In [1]:
from sentence_transformers import SentenceTransformer, util
import torch

c:\Users\jjsd4\miniconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\jjsd4\miniconda3\envs\torch_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jjsd4\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

In [3]:
# 예시 문장
sentences = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin."
]

# 임베딩화
embeddings = model.encode(sentences)

In [6]:
embeddings.shape

(5, 384)

In [5]:
embeddings

array([[ 0.03324157,  0.00440613, -0.00627694, ...,  0.06917296,
        -0.02458482, -0.03756436],
       [ 0.05251843,  0.05524132, -0.0111882 , ..., -0.01619389,
        -0.060229  , -0.04115056],
       [-0.03630894, -0.03565347, -0.02722264, ..., -0.03858706,
         0.10574473, -0.00131469],
       [-0.00462491, -0.01413293,  0.04100329, ...,  0.00743566,
         0.06817716, -0.02425219],
       [-0.02668153, -0.0987884 ,  0.028563  , ...,  0.02552059,
         0.09566616, -0.00743377]], dtype=float32)

In [7]:
# 예시문장 유사도 계산
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
print("문장 유사도:", similarity.item())

문장 유사도: 0.755337119102478


In [8]:
# 예시문장 유사도 계산
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[3])
print("문장 유사도:", similarity.item())

문장 유사도: 0.2474014163017273


In [9]:
# 예시문장 유사도 계산
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[4])
print("문장 유사도:", similarity.item())

문장 유사도: -0.07038549333810806


In [11]:
# 클러스터링
from sklearn.cluster import KMeans

num_clusters = 2

clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

for i in range(num_clusters):
    print(f"\nCluster {i+1}")
    cluster_sentences = [sentences[j] for j in range(len(sentences)) if cluster_assignment[j] == i]
    for sentence in cluster_sentences:
        print(sentence)


Cluster 1
A man is eating food.
A man is eating a piece of bread.
A man is riding a horse.

Cluster 2
The girl is carrying a baby.
A woman is playing violin.


In [12]:
# 클러스터링
from sklearn.cluster import KMeans

num_clusters = 3

clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

for i in range(num_clusters):
    print(f"\nCluster {i+1}")
    cluster_sentences = [sentences[j] for j in range(len(sentences)) if cluster_assignment[j] == i]
    for sentence in cluster_sentences:
        print(sentence)


Cluster 1
The girl is carrying a baby.
A woman is playing violin.

Cluster 2
A man is eating food.
A man is eating a piece of bread.

Cluster 3
A man is riding a horse.


In [13]:
# 쿼리 예시
sentences = [
    "우리나라 대통령 중 4대 대통령 전 대통령이 누구야?", #질문 문장
    "대한민국 1대 대통령은 이승만이다.", #대상 문장
    "대한민국 2대 대통령은 이승만이다.", #대상 문장
    "대한민국 3대 대통령은 이승만이다.", #대상 문장
    "대한민국 4대 대통령은 윤보선이다.", #대상 문장
]

embeddings = model.encode(sentences)

In [14]:
# 쿼리에 대한 유사도 계산
similarity = util.pytorch_cos_sim(embeddings[0],embeddings[1])
print("문장 유사도:", similarity.item())

문장 유사도: 0.568516731262207


In [15]:
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[2])
print("문장 유사도:", similarity.item())

문장 유사도: 0.5799103379249573


In [16]:
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[3])
print("문장 유사도:", similarity.item())

문장 유사도: 0.5694818496704102


In [ ]:
similarity = util.pytorch_cos_sim(embeddings[0], embeddings[4])
print("문장 유사도:", similarity.item())

문장 유사도: 0.7418656349182129


#명확하게 사용자가 질문을 하지 않으면 (사용자가 실제로 애매한 질문을 하는 경우가 상당히 많음)
-> 우리가 구성한 프로그램이 제대로 된 답변을 내놓지 않는다고 생각함